In [ ]:
# ===== Đoạn code của bạn giữ nguyên bên dưới =====
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd

# Đường dẫn geckodriver
gecko_path = r"C:\Users\HP\Documents\Downloads\geckodriver-v0.35.0-win64\geckodriver.exe"
service = Service(gecko_path)
driver = webdriver.Firefox(service=service)

# Danh sách rapper
rapper_list = [
    "Seachains", "Pháo", "Kellie",
    "Lona", "Wean", "Táo", "Cam", "BigDaddy", "Emily", "Datmaniac", "Blacka",
    "95G", "Kimmese", "Lil Wuyn", "Tommy Tèo", "Tez", "Captain", "Evy"
]

# XPaths
search_input_xpath = '//*[@id="center"]/yt-searchbox/div[1]/form/input'
channel_xpath = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-channel-renderer/div/div[2]/a/div[1]/ytd-channel-name/div/div/yt-formatted-string'
video_tab_xpath = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse[2]/div[4]/ytd-tabbed-page-header/tp-yt-app-header-layout/div/tp-yt-app-header/div[2]/tp-yt-app-toolbar/div/div/tp-yt-paper-tabs/div/div/yt-tab-group-shape/div[1]/yt-tab-shape[2]/div[1]'
like_xpath = '//*[@id="top-level-buttons-computed"]/segmented-like-dislike-button-view-model/yt-smartimation/div/div/like-button-view-model/toggle-button-view-model/button-view-model/button/div[2]'

all_data = []  # lưu dữ liệu của tất cả rapper

for rapper in rapper_list:
    print(f"\n===== ĐANG CÀO RAPPER: {rapper} =====")

    # -------- B1: mở YouTube và tìm kiếm ----------
    driver.get("https://www.youtube.com/")
    time.sleep(6)

    search_input = driver.find_element(By.XPATH, search_input_xpath)
    search_input.clear()
    search_input.send_keys(rapper)
    search_input.send_keys(Keys.ENTER)
    time.sleep(10)

    # -------- B2: tìm và click vào Kênh ----------
    found_channel = False
    while not found_channel:
        try:
            driver.find_element(By.XPATH, channel_xpath).click()
            found_channel = True
        except:
            driver.execute_script("window.scrollBy(0, 400);")
            time.sleep(5)
    time.sleep(3)

    # -------- B3: click tab Video ----------
    clicked_videos_tab = False
    while not clicked_videos_tab:
        try:
            driver.find_element(By.XPATH, video_tab_xpath).click()
            clicked_videos_tab = True
        except:
            driver.execute_script("window.scrollBy(0, 300);")
            time.sleep(1)
    time.sleep(3)

    # -------- B4: duyệt từng video ----------
    index = 1
    max_missing = 20

    while True:
        try:
            xpath_video_block = f'(//ytd-rich-grid-media)[{index}]'
            video_elem = driver.find_element(By.XPATH, xpath_video_block)

            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", video_elem)
            time.sleep(2)

            # --- lấy thông tin ở ngoài ---
            try:
                title = video_elem.find_element(By.XPATH, './/h3/a/yt-formatted-string').text
            except:
                title = "N/A"
            try:
                views = video_elem.find_element(By.XPATH, './/ytd-video-meta-block/div[1]/div[2]/span[1]').text
            except:
                views = "N/A"
            try:
                time_posted = video_elem.find_element(By.XPATH, './/ytd-video-meta-block/div[1]/div[2]/span[2]').text
            except:
                time_posted = "N/A"

            # --- click vào video để lấy Like ---
            try:
                thumb = video_elem.find_element(By.XPATH, './/a[@id="thumbnail"]')
                thumb.click()
                time.sleep(3)
                try:
                    likes = driver.find_element(By.XPATH, like_xpath).text
                except:
                    likes = "N/A"
            except:
                likes = "N/A"

            # Lưu kết quả
            all_data.append({
                "Rapper": rapper,
                "Tên Clip": title,
                "Lượt xem": views,
                "Thời gian đăng": time_posted,
                "Like": likes
            })
            print(f"Đã lấy video {index}: {title} | {views} | {time_posted} | {likes}")

            # Quay lại tab Video
            driver.back()
            time.sleep(3)

            # cuộn nhẹ để chắc chắn load video tiếp theo
            driver.execute_script("window.scrollBy(0, 200);")
            time.sleep(2)

            index += 1

        except NoSuchElementException:
            # không thấy video index hiện tại → thử cuộn
            driver.execute_script("window.scrollBy(0, 600);")
            time.sleep(2)
            max_missing -= 1
            if max_missing <= 0:
                print(f"Không còn video mới nào cho {rapper}.")
                break

# Xuất ra Excel sau khi cào hết tất cả rapper
df = pd.DataFrame(all_data)
df.to_excel("youtube_all_rappers.xlsx", index=False, engine="openpyxl")
print("Đã lưu danh sách video của tất cả rapper vào youtube_all_rappers.xlsx")
# Đóng trình duyệt
driver.quit()